In [4]:
import tdwf
import matplotlib
import numpy as np
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import time

# -[MPU6050 - Qualche comando]---------------------------------------------------
PWR_MGMT1 = 0x6B
ACCEL_CONFIG = 0x1C
GYRO_CONFIG = 0x1B


TEMP_OUT_H = [0x41]
TEMP_OUT_L = [0x42]

ACCEL_XOUT_H = [0x3B]
ACCEL_XOUT_L = [0x3C]
ACCEL_YOUT_H = [0x3D]
ACCEL_YOUT_L = [0x3E]
ACCEL_ZOUT_H = [0x3F]
ACCEL_ZOUT_L = [0x40]

GYRO_XOUT_H = [0x43]
GYRO_XOUT_L = [0x44]
GYRO_YOUT_H = [0x45]
GYRO_YOUT_L = [0x46]
GYRO_ZOUT_H = [0x47]
GYRO_ZOUT_L = [0x48]

SAD = 0x68
g = 9.80665

# -[Configurazione AD2]--------------------------------------------------------
#   1. Connessiene con AD2 e selezione configurazione
ad2 = tdwf.AD2()
#   2. Configurazione alimentazione
ad2.vdd = 3.3
ad2.power(True)
#   3. Impostazione bus I2C
i2c = tdwf.I2Cbus(ad2.hdwf)  # default 100kHz, SCL = D00, SDA = D01
devs = i2c.scan()  # verifica cosa è connesso...
for dev in devs:
    print(f"Device: 0x{dev:02x}")
#   4. Apre la comunicazione I2C come Master
sht = tdwf.I2Cdevice(ad2.hdwf,SAD)

# -[Configurazione sensore]------------------------------------------------------
sht.write([PWR_MGMT1, 0x00]) # Accende il sensore
sht.write([ACCEL_CONFIG, 0x03]) # Accel a 2g
sht.write([GYRO_CONFIG, 0x08]) # Gyro a 1000dps

# -[Funzioni di gestione eventi]-----------------------------------------------

def on_close(event):
    global flag_run
    flag_run = False

def on_key(event):
    global flag_run
    if event.key == 'escape':  # termina programma
        flag_run = False
    if event.key == 'x':
        filename = input('gimme gimme gimme a name to save data: ')
        info = f"time\tTemp\tAccX\tAccY\tAccZ\tGyrX\tGyrY\tGyrZ"
        np.savetxt("../Data/"+filename+".txt", np.c_[mytime, TTv, AXv, AYv, AZv, GXv, GYv, GZv], header=info)
        print("Data saved in Data/"+filename+".txt")

# -[Funzioni di supporto]-------------------------------------------------------
def read2bytes(addr):
    # Read two consecutive bytes
    sht.writeread(addr,1)
    BH = sht.vals[0]
    sht.writeread([addr[0]+1],1)
    BL = sht.vals[0]
    # 2's complement
    BT = (BH *256.0 ) + BL
    if BT>0x8000:
        BT-=0x10000
    return BT


# -[Ciclo di misura]-----------------------------------------------------------
#   1. Creazione figura e link agli eventi
fig, axs = plt.subplots(2, 1, figsize=(12,6))
fig.canvas.mpl_connect("close_event", on_close)
fig.canvas.mpl_connect("key_press_event", on_key)

#   2. Inizializzazione variabili
mytime = []
TTv = []

AXv = []
AYv = []
AZv = []

GXv = []
GYv = []
GZv = []


flag_first = True
flag_run = True

start_time = time.time()

#   3. Ciclo di misura
while flag_run:
#    time.sleep(0.001)

    TT = read2bytes(TEMP_OUT_H)
    TT = TT/340 + 36.53 # Conversione temperatura
    TTv.append(TT)
    print(f"T = {TT:.2f}C")

    AX = read2bytes(ACCEL_XOUT_H)
    AY = read2bytes(ACCEL_YOUT_H)
    AZ = read2bytes(ACCEL_ZOUT_H)
    AX = AX/16384.0 * g
    AY = AY/16384.0 * g
    AZ = AZ/16384.0 * g
    AXv.append(AX)
    AYv.append(AY)
    AZv.append(AZ)

    GX = read2bytes(GYRO_XOUT_H)
    GY = read2bytes(GYRO_YOUT_H)
    GZ = read2bytes(GYRO_ZOUT_H)
    GX = GX/32.768
    GY = GY/32.768
    GZ = GZ/32.768
    GXv.append(GX)
    GYv.append(GY)
    GZv.append(GZ)

    mytime.append(time.time()-start_time)
    if flag_first:
        flag_first = False
#        hp1, = plt.plot(mytime,TTv, "o", color="tab:orange")
        hp2, = axs[0].plot(mytime,AXv, "o-", color="tab:red")
        hp3, = axs[0].plot(mytime,AYv, "o-", color="tab:green")
        hp4, = axs[0].plot(mytime,AZv, "o-", color="tab:blue")

        hp5, = axs[1].plot(mytime,GXv, "o-", color="tab:pink")
        hp6, = axs[1].plot(mytime,GYv, "o-", color="tab:olive")
        hp7, = axs[1].plot(mytime,GZv, "o-", color="tab:cyan")



        axs[0].grid(True)
        axs[0].set_ylabel("Acc [m/s^2]")
#        axs[0].set_xlabel("Time [s]")
#        axs[0].title("Press ESC to exit")

        axs[1].grid(True)
        axs[1].set_ylabel("Angular Velocity [°/s]")
        axs[1].set_xlabel("Time [s]")
        plt.show(block=False)    
        plt.tight_layout()
    else:
#        hp1.set_ydata(TTv)
#        hp1.set_xdata(mytime)
        hp2.set_ydata(AXv)
        hp3.set_ydata(AYv)
        hp4.set_ydata(AZv)
        hp2.set_xdata(mytime)
        hp3.set_xdata(mytime)
        hp4.set_xdata(mytime)

        hp5.set_ydata(GXv)
        hp6.set_ydata(GYv)
        hp7.set_ydata(GZv)
        hp5.set_xdata(mytime)
        hp6.set_xdata(mytime)
        hp7.set_xdata(mytime)


        axs[0].relim()           
        axs[0].autoscale_view() 
        axs[1].relim()
        axs[1].autoscale_view()
        fig.canvas.draw()
        fig.canvas.flush_events()
# ---------------------------------------
plt.close(fig)
ad2.close()




Dispositivo #1 [SN:210321ABE62D, hdwf=1] connesso!
Configurazione #1
Bus I2C pronto...
Device: 0x68
T = 32.01C
T = 32.11C
T = 32.20C
T = 32.20C
T = 32.25C
T = 32.15C
T = 32.25C
T = 32.25C
T = 32.34C
T = 32.29C
T = 32.25C
T = 32.29C
T = 32.25C
T = 32.29C
T = 32.25C
T = 32.44C
T = 32.34C
T = 32.34C
T = 32.39C
T = 32.39C
T = 32.39C
T = 32.29C
T = 32.44C
T = 32.39C
T = 32.44C
T = 32.39C
T = 32.34C
T = 32.39C
T = 32.34C
T = 32.44C
T = 32.44C
T = 32.58C
T = 32.48C
T = 32.53C
T = 32.53C
T = 32.58C
T = 32.53C
T = 32.44C
T = 32.53C
T = 32.48C
T = 32.53C
T = 32.44C
T = 32.53C
T = 32.48C
T = 32.44C
T = 32.58C
T = 32.44C
T = 32.53C
T = 32.58C
T = 32.48C
T = 32.53C
T = 32.58C
T = 32.58C
T = 32.48C
T = 32.58C
T = 32.53C
T = 32.62C
T = 32.62C
T = 32.53C
T = 32.44C
T = 32.53C
T = 32.58C
T = 32.53C
T = 32.62C
T = 32.62C
T = 32.53C
T = 32.58C
T = 32.67C
T = 32.58C
T = 32.58C
T = 32.53C
T = 32.58C
T = 32.58C
T = 32.62C
T = 32.72C
T = 32.62C
T = 32.62C
T = 32.58C
T = 32.72C
T = 32.67C
T = 32.53C
T = 32.62